Initialize

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import time
import csv
from alive_progress import alive_bar
from scripts.BinanceClient import BinanceClient
from scripts import database

bc = BinanceClient(futures=True)

1. Get selected symbols

In [24]:
symbols = database.read("symbols.csv")
print(symbols)

['RENUSDT', 'RSRUSDT', 'LINAUSDT', 'TRXUSDT', 'IOSTUSDT', 'BNBUSDT', 'DYDXUSDT', 'SOLUSDT', 'STORJUSDT', 'BTCUSDT', 'IOTAUSDT', 'DOGEUSDT', 'C98USDT', 'XRPUSDT', 'ATAUSDT', 'XLMUSDT', 'IOTXUSDT', 'ADAUSDT', 'NEARUSDT', 'CRVUSDT', 'BCHUSDT', 'DASHUSDT', 'ETHUSDT', 'KAVAUSDT', 'FTMUSDT', 'OGNUSDT', 'COTIUSDT', 'LINKUSDT', 'ANKRUSDT', 'DOTUSDT']


2. Download historical data

In [79]:
limit_candles = 1500
interval = 60_000  # 1min interval
delta = limit_candles * interval  # time difference between candles

days_back = 1000
total_candles = days_back * 24 * (interval // 1000)  # total candles to be collected

date_loaded = False
end_date = bc.current_time()
initial_date = end_date - (days_back * 24 * 3600 * 1000)  # {days_back} days behind

i = 0
for symbol in symbols:
    i += 1
    with alive_bar(total_candles, force_tty=True, title=f'{i} / {len(symbols)} {symbol}') as bar:
        start_time = initial_date
        num_rows = 0  # rows including header

        file_name = f'{symbol}.csv'
        if database.exits(file_name):
            with open(database.to_db_path(file_name), 'r') as file:
                for row in csv.reader(file):
                    if not date_loaded and num_rows == 1:
                        # set to the first run date
                        initial_date = int(float(row[0]))
                        end_date = initial_date + (days_back * 24 * 3600 * 1000)
                        date_loaded = True
                    num_rows += 1

            if num_rows > total_candles:  # data already collected
                continue

            if num_rows > 1:  # more than header
                start_time = int(float(row[0])) + interval  # continue from the last timestamp
        else:
            database.write(file_name, ['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            num_rows = 1

        bar(num_rows - 1)

        while num_rows < total_candles + 1:  # 1 for header row
            end_time = start_time + delta

            symbol_data = bc.get_historical_data(
                symbol,
                interval='1m',
                start_time=start_time,
                end_time=end_time,
            )

            for data in symbol_data:
                database.write(file_name, data)
                num_rows += 1

            start_time = int(data[0]) + interval
            bar(len(symbol_data))
            time.sleep(1)

        print(f'{symbol} done!')
        time.sleep(2)

KeyboardInterrupt: 